In [1]:
a  = [0]*3

In [3]:
list(range(5, 1, -1))

[5, 4, 3, 2]

In [11]:
b = (-1).to_bytes(2, 'big', signed=True)

In [12]:
b[0]

255

In [6]:
b'\xff\xff' 

b'\xff\xff'

In [8]:
class Cint:
    def __init__(self, n, size=32, signed=True):
        self.n = n % (1 << size)
        self.size = size
        self.signed = signed
    def __add__(x, y):
        size = max(x.size, y.size)
        z = (x + y) % (1 << size)
        if x.size > y.size:
            signed = x.signed
        elif y.size > x.size:
            signed = y.signed
        elif x.signed == y.signed:
            signed = x.signed
        else:
            signed = False

0.01384128720099999

In [8]:
INTSIZE = 32
def int16_negative_mask(x):
    u = 0
    if x < 0:
        u = -1
    return u.to_bytes(INTSIZE/8, 'big', signed=True)
def int16_nonzero_mask(x):
    u = 0
    if x != 0:
        u = -1
    return u.to_bytes(INTSIZE/8, 'big', signed=True)
def F3_freeze(x):
    

In [ ]:
def R3_recip(inp, p=653):
    v = [0]*(p+1)
    r = [0]*(p+1)
    r[0] = 1
    f = [0]*(p+1);
    f[0] = 1
    f[p-1] = f[p] = -1;
    g = [inp[p-1-j] for j in range(p)]
    g[p] = 0;
    
    delta = 1; 

    for loop in range(2*p-1): 
        for i in range(p, 0, -1):
            v[i] = v[i-1]
        v[0] = 0
        
        sign = -g[0]*f[0]
        
        a = int16_negative_mask(-delta)
        b = int16_nonzero_mask(g[0])
        swap = bytes([a[i] & b[i] for i in range(INTSIZE/8)])
        
        a = delta.to_bytes(INTSIZE/8, 'big', signed=True)
        b = (-delta).to_bytes(INTSIZE/8, 'big', signed=True)
        db = bytes([ a[i] ^ ( swap[i] & (a[i] ^ b[i]) ) for i in range(INTSIZE/8)])
        delta = (int.from_bytes(db, 'big') + 1) % (1 << INTSIZE)
        if delta > (1 << (INTSIZE - 1)):
            delta = delta - (1 << INTSIZE)
        
        for i in range(p+1):
            fi = f[i].to_bytes(INTSIZE/8, 'big', signed=True)
            gi = g[i].to_bytes(INTSIZE/8, 'big', signed=True)
            t = bytes([swap[j] & (fi[j] ^ gi[j]) for j in range(INTSIZE/8)])
            f[i] = ( [fi[j] ^ t[j] for j in range(INTSIZE/8)] )
            f[i] = ( [fi[j] ^ t[j] for j in range(INTSIZE/8)] )

            vi = v[i].to_bytes(INTSIZE/8, 'big', signed=True)
            ri = r[i].to_bytes(INTSIZE/8, 'big', signed=True)
            t = bytes( [swap[j] & (vi[j] ^ ri[j]) for j in range(INTSIZE/8)] )
            v[i] = ( [vi[j] ^ t[j] for j in range(INTSIZE/8)] )
            r[i] = ( [ri[j] ^ t[j] for j in range(INTSIZE/8)] )

        for i in range(p+1):
            g[i] = F3_freeze(g[i]+sign*f[i])
            r[i] = F3_freeze(r[i]+sign*v[i])
            

        for i in range(p):
            g[i] = g[i+1]
        g[p] = 0;

    sign = f[0];
    for (i = 0;i < p;++i) out[i] = sign*v[p-1-i];

    return int16_nonzero_mask(delta);